In [6]:
from common_imports import *
show_home_button()
from db_connection import get_engine
engine = get_engine()

Output()

2025-05-16 10:37:42,869 | INFO     | SQL-engine aangemaakt voor inn-vee-sql12/EDS2


In [7]:
env_file = Path.cwd() / ".env"
if env_file.exists():
    from dotenv import load_dotenv
    load_dotenv(env_file)

os.environ.setdefault("DB_HOST", "inn-vee-sql12")
os.environ.setdefault("DB_DATABASE", "EDS2")          

engine = get_engine()                

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s | %(levelname)-8s | %(message)s")


2025-05-16 10:37:42,943 | INFO     | SQL-engine aangemaakt voor inn-vee-sql12/EDS2


In [8]:
class DatabaseManager:
    """
    Databasema­nager op basis van één gedeelde SQLAlchemy-engine
    (autocommit=True – identiek aan de oude pyodbc-verbindingen).
    """
    def __init__(self, engine):
        self.engine = engine

    def _connect(self):
        return self.engine.raw_connection()

    # ---------- reads ----------
    def fetch_registers(self, registrator_id: int):
        q = text("""
            SELECT ID AS RegisterId
            FROM dbo.TBL_Register
            WHERE RegistratorId = :rid
            ORDER BY ID;
        """)
        return pd.read_sql(q, self.engine, params={"rid": registrator_id})["RegisterId"].tolist()

    def fetch_register_settings(self, register_ids: list[int]):
        if not register_ids:
            return pd.DataFrame(columns=["RegisterId", "CollectInterval", "StorageMethodId"])
        ph = ",".join(":id" + str(i) for i in range(len(register_ids)))
        q = text(f"""
            SELECT ID AS RegisterId, CollectInterval, StorageMethodId
            FROM dbo.TBL_Register
            WHERE ID IN ({ph})
            ORDER BY ID;
        """)
        params = {f"id{i}": v for i, v in enumerate(register_ids)}
        return pd.read_sql(q, self.engine, params=params)

    def _get_method_id(self, minutes: int) -> int:
        q = text("""
            SELECT Id
            FROM dbo.TBL_Ref_Register_StorageMethod
            WHERE CollectInterval = :min;
        """)
        df = pd.read_sql(q, self.engine, params={"min": minutes})
        if df.empty:
            raise ValueError(f"Geen StorageMethod voor interval {minutes}")
        return int(df.loc[0, "Id"])

    # ---------- writes ----------
    def update_registers(self, register_ids: list[int], minutes: int):
        if not register_ids:  # nothing to do
            return
        method_id = self._get_method_id(minutes)
        ph = ",".join(":id" + str(i) for i in range(len(register_ids)))
        sql = text(f"""
            UPDATE dbo.TBL_Register
            SET CollectInterval = :minutes, StorageMethodId = :method
            WHERE ID IN ({ph});
        """)
        params = {"minutes": minutes, "method": method_id} | {f"id{i}": v for i, v in enumerate(register_ids)}
        with self.engine.connect() as conn:       # autocommit=True → direct commit
            conn.execute(sql, params)
        logging.info("Registers %s geüpdatet → %d min, StorageMethodId=%d",
                     register_ids, minutes, method_id)

    def run_repair(self, registrator_id: int, start_local: str, end_local: str, do_update: bool):
        """
        Roept de uitgebreide T-SQL repair-routine aan.
        """
        sql = text(r"""
        EXECUTE dbo.FixRegisterStorage
            @RegistratorId      = :rid,
            @StartLocal         = :start,
            @EndLocal           = :end,
            @DoUpdate           = :flag;
        """)
        with self.engine.connect() as conn:        # autocommit=True
            conn.execute(sql, {"rid": registrator_id,
                               "start": start_local,
                               "end": end_local,
                               "flag": 1 if do_update else 0})
        logging.info("Repair uitgevoerd voor RegistratorId=%d  (DoUpdate=%s)",
                     registrator_id, do_update)

In [9]:
class UIManager:
    def __init__(self, db: DatabaseManager):
        self.db = db
        self.reg_ids: list[int] = []
        self.fixed_end = "2026-01-01 00:00:00"
        self._build_ui()

    def _build_ui(self):
        self.out = widgets.Output()

        self.reg_input_label = widgets.Label(
            "RegistratorId:",
            layout=widgets.Layout(display='flex', align_items='center')
        )
        self.reg_input = widgets.Text(
            description='',                
            placeholder='Vul RegistratorId in',  
            value='', 
        layout=widgets.Layout(width='100px')
        )
        self.fetch_btn = widgets.Button(
            description='Fetch RegisterIDs',
            icon='database',
            tooltip='Fetch register IDs and their current settings'
        )
        self.run_btn = widgets.Button(
            description='Run Repair',
            icon='play',
            disabled=True,
            tooltip='Run the repair process with the specified settings'
        )
        self.run_btn.add_class('mod-success')

        # Row 1: RegistratorId input, Fetch button, Run Repair button
        toolbar_row = widgets.HBox(
            [self.reg_input_label, self.reg_input, self.fetch_btn, self.run_btn],
            layout=widgets.Layout(align_items='center', gap='8px')
        )

        self.start_local_label = widgets.Label("StartLocal:", layout=widgets.Layout(display='flex', align_items='center'))
        self.start_local = widgets.Text(
            value="2024-01-01 00:05", # Default or example value
            description='',
            layout=widgets.Layout(width='180px')
        )
        self.end_local_html = widgets.HTML(
            f"<b>EndLocal:</b> {self.fixed_end}",
            layout=widgets.Layout(display='flex', align_items='center')
        )
        self.minutes_label = widgets.Label("Minutes:", layout=widgets.Layout(display='flex', align_items='center'))
        self.interval_dropdown = widgets.Dropdown(
            options=[("5 minuten",5), ("15 minuten",15)],
            value=15,
            description='',
            layout=widgets.Layout(width='130px')
        )
        self.do_update = widgets.Checkbox(
            value=True,
            description='DoUpdate',
            indent=False,
            layout=widgets.Layout(display='flex', align_items='center')
        )
        form_fields_row = widgets.HBox(
            [self.start_local_label, self.start_local, self.end_local_html,
             self.minutes_label, self.interval_dropdown, self.do_update],
            layout=widgets.Layout(align_items='center', flex_wrap='nowrap', gap='8px')
        )

        self.settings_df_title = widgets.HTML("<b>Current settings:</b>")
        self.settings_df = widgets.Output(layout=widgets.Layout(width='100%'))

        # Overall layout container
        controls_box = widgets.VBox(
            [toolbar_row, form_fields_row, self.settings_df_title, self.settings_df],
            layout=widgets.Layout(
                padding='8px', border='1px solid #cccccc', border_radius='4px',
                gap='12px' # Vertical gap between rows and table title
            )
        )

        self.fetch_btn.on_click(self.on_fetch)
        self.run_btn.on_click(self.on_run)

        self.tabs = widgets.Tab(layout=widgets.Layout(margin='10px 0 0 0'))
        display(controls_box, self.out, self.tabs)

    def on_fetch(self, _):
        with self.out:
            clear_output(wait=True)
            print(f"Fetching registers for RegistratorId: {self.reg_input.value}...")
        try:
            registrator_id_val = self.reg_input.value
            if registrator_id_val is None: # Or handle empty string if IntText allows
                with self.out:
                    clear_output(wait=True)
                    print("RegistratorId cannot be empty.")
                self.run_btn.disabled = True
                with self.settings_df:
                    clear_output(wait=True)
                return

            regs = self.db.fetch_registers(registrator_id_val)
            self.reg_ids = regs

            with self.out:
                clear_output(wait=True)
                if regs:
                    print(f"Successfully fetched. Register IDs: {', '.join(map(str, regs))}")
                else:
                    print(f"No registers found for RegistratorId: {registrator_id_val}.")

            df = self.db.fetch_register_settings(regs)
            with self.settings_df:
                clear_output(wait=True)
                if not df.empty:
                    styles = [
                        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', 'auto'), ('border', 'none')]},\
                        {'selector': 'th', 'props': [
                            ('text-align', 'left'), ('font-weight', 'bold'), ('background-color', '#f0f0f0'), ('padding', '8px'),
                            ('border-bottom', '1px solid #ddd')
                        ]},\
                        {'selector': 'td', 'props': [
                            ('text-align', 'left'), ('padding', '8px'),
                            ('border-bottom', '1px solid #eee')
                        ]},\
                        {'selector': 'tr:nth-child(even) td', 'props': [('background-color', '#f9f9f9')]},\
                        {'selector': 'tr:nth-child(odd) td', 'props': [('background-color', '#ffffff')]},\
                        {'selector': 'th, td', 'props': [('border-left', 'none'), ('border-right', 'none'), ('border-top', 'none')]}
                    ]
                    df_styled = df.style.set_table_styles(styles).hide(axis="index")
                    display(HTML(df_styled.to_html()))
                elif regs: # Registers found, but settings DataFrame is empty
                    print("Registers found, but no specific settings retrieved.")
                else: # No registers found, message already in self.out
                    pass 

            self.run_btn.disabled = not bool(regs)

        except Exception as e:
            with self.out:
                clear_output(wait=True)
                logging.error(f"Error during fetch: {e}")
                print(f"An error occurred: {e}")
            self.run_btn.disabled = True
            with self.settings_df:
                clear_output(wait=True)


    def on_run(self, _):
        with self.out:
            clear_output(wait=True)
            print(f"Preparing to run repair for RegistratorId: {self.reg_input.value} with interval {self.interval_dropdown.value} minutes.")
            print("Starting pre-update and repair…")
        try:
            mins = self.interval_dropdown.value
            self.db.update_registers(self.reg_ids, mins)
            with self.out: 
                 current_content = self.out.outputs[0]['data']['text/plain'] if self.out.outputs else ""
                 clear_output(wait=True)
                 print(current_content.strip() + f"\nRegisters {self.reg_ids} successfully set to {mins} minutes interval.\nProceeding with the main repair process...")


            df_log, df_pivot = self.db.run_repair(
                self.reg_input.value,
                self.start_local.value,
                self.fixed_end,
                self.do_update.value
            )
            with self.out:
                clear_output(wait=True)
                print("Repair process completed.")
                if self.do_update.value:
                    print("Data update was performed.")
                else:
                    print("Data update was skipped as per selection.")

            self.tabs.children = [widgets.Output(), widgets.Output()]
            self.tabs.set_title(0, 'Repair Log')
            self.tabs.set_title(1, 'Pivot Data')

            with self.tabs.children[0]:
                clear_output(wait=True)
                if not df_log.empty:
                    display(df_log)
                else:
                    print("Repair log is empty.")

            with self.tabs.children[1]:
                clear_output(wait=True)
                if not df_pivot.empty:
                    display(df_pivot)
                else:
                    print("Pivot data is empty.")

            self.on_fetch(None) # Refresh settings display

        except Exception as e:
            with self.out:
                clear_output(wait=True)
                logging.error(f"Error during run: {e}")
                print(f"An error occurred during the repair process: {e}")

In [10]:
dbm = DatabaseManager(engine)
ui  = UIManager(dbm)           

Output()

Tab(layout=Layout(margin='10px 0 0 0'))